# SMAI Assignment 4 - Question 3: Trees and Random Forests

This notebook covers **Part 2 (Decision Trees)** and **Part 3 (Random Forests)** using `scikit-learn`.

### Imports

This cell imports all necessary libraries and also includes the `pretty_print_sklearn_tree` helper function.

In [1]:
import numpy as np
import pandas as pd
import os
import time
import sklearn.tree
import sklearn.metrics
import sklearn.ensemble
import sklearn.model_selection
from collections import defaultdict
import warnings

# Suppress sklearn warnings for cleaner output
warnings.filterwarnings("ignore", category=UserWarning)

# --- Utility Function (from pretty_print_sklearn_tree.py) ---
def pretty_print_sklearn_tree(tree_clf, feature_names):
    ''' Print out a nice summary of the provided tree.'''
    n_nodes = tree_clf.tree_.node_count
    children_left = tree_clf.tree_.children_left
    children_right = tree_clf.tree_.children_right
    feature = tree_clf.tree_.feature
    threshold = tree_clf.tree_.threshold

    node_depth_N = np.zeros(shape=n_nodes, dtype=np.int64)
    is_leaf_N = np.zeros(shape=n_nodes, dtype=bool)
    stack = [(0, -1)]
    while len(stack) > 0:
        node_id, parent_depth = stack.pop()
        node_depth_N[node_id] = parent_depth + 1
        if (children_left[node_id] != children_right[node_id]):
            stack.append((children_left[node_id], parent_depth + 1))
            stack.append((children_right[node_id], parent_depth + 1))
        else:
            is_leaf_N[node_id] = True

    print("The binary tree structure has %s nodes." % n_nodes)
    depths_U, counts_U = np.unique(node_depth_N, return_counts=True)
    for uu in range(depths_U.size):
        is_at_cur_depth_N = depths_U[uu] == node_depth_N
        is_leaf_at_cur_depth_N = np.logical_and(is_leaf_N, is_at_cur_depth_N)
        print("- depth %3d has %4d nodes, of which %4d are leaves" % (
            depths_U[uu], counts_U[uu], np.sum(is_leaf_at_cur_depth_N)))

    print("The decision tree:  (Note: Y = 'yes' to above question; N = 'no')")
    n_seen_by_depth = defaultdict(int)
    for i in range(n_nodes):
        cur_depth = node_depth_N[i]
        count_at_cur_depth = n_seen_by_depth[cur_depth]
        if node_depth_N[i] == 0:
            decision_str = ''
        elif count_at_cur_depth % 2 == 0:
            decision_str = 'Y '
        else:
            decision_str = 'N '
        if is_leaf_N[i]:
            n_class0 = tree_clf.tree_.value[i,0,0]
            n_class1 = tree_clf.tree_.value[i,0,1]
            if (n_class1 + n_class0) == 0:
                proba1 = 0.0
            else:
                proba1 = n_class1 / (n_class1 + n_class0)
            print("%s%sLeaf: p(y=1 | this leaf) = %.3f (%d total training examples)" % (
                node_depth_N[i] * "  ", decision_str, proba1, n_class0 + n_class1))
        else:
            print("%s%sDecision: X['%s'] <= %.2f?" % (
                node_depth_N[i] * "  ",
                decision_str,
                feature_names[feature[i]],
                threshold[i],
                ))
        n_seen_by_depth[cur_depth] += 1
    print()

### Load Data

**IMPORTANT:** This notebook assumes your data is in a sub-folder named `data_product_reviews`.

In [2]:
print("Loading data...")
DATA_DIR = 'data_product_reviews'

try:
    # --- THIS BLOCK IS UPDATED TO USE PANDAS --- 
    # Pandas gracefully handles CSVs with headers.
    
    # Load x_train and get vocabulary from its header
    df_x_tr = pd.read_csv(os.path.join(DATA_DIR, 'x_train.csv'))
    vocab_list = df_x_tr.columns.values
    x_tr_NF = df_x_tr.values.astype(np.float64)
    
    # Load y_train
    df_y_tr = pd.read_csv(os.path.join(DATA_DIR, 'y_train.csv'))
    y_tr_N = df_y_tr.values.squeeze().astype(np.int64) # .squeeze() to make it 1D

    # Load validation data
    x_va_NF = pd.read_csv(os.path.join(DATA_DIR, 'x_valid.csv')).values.astype(np.float64)
    y_va_N = pd.read_csv(os.path.join(DATA_DIR, 'y_valid.csv')).values.squeeze().astype(np.int64)

    # Load test data
    x_te_NF = pd.read_csv(os.path.join(DATA_DIR, 'x_test.csv')).values.astype(np.float64)
    y_te_N = pd.read_csv(os.path.join(DATA_DIR, 'y_test.csv')).values.squeeze().astype(np.int64)
    # --- END OF UPDATE ---

    print("Data loaded successfully.")
    print(f"x_train shape: {x_tr_NF.shape}")
    print(f"y_train shape: {y_tr_N.shape}")
    print(f"x_valid shape: {x_va_NF.shape}")
    print(f"x_test shape: {x_te_NF.shape}")
    print(f"Vocabulary size: {len(vocab_list)}")

except Exception as e:
    print(f"Error loading data: {e}")
    print(f"Please make sure all .csv files are in the '{DATA_DIR}' folder.")

Loading data...
Data loaded successfully.
x_train shape: (6346, 7729)
y_train shape: (6346,)
x_valid shape: (792, 7729)
x_test shape: (793, 7729)
Vocabulary size: 7729


### Prepare Data for GridSearchCV

We need to combine the train and validation sets for `GridSearchCV` and create a special splitter object that tells it to use our specific train/validation split.

In [3]:
# Pack training and validation sets for GridSearchCV
x_tr_va_NF = np.vstack([x_tr_NF, x_va_NF])
y_tr_va_N = np.hstack([y_tr_N, y_va_N])

n_tr = x_tr_NF.shape[0]
n_va = x_va_NF.shape[0]
tr_va_inds = np.arange(n_tr + n_va)
tr_inds = tr_va_inds[:n_tr]
va_inds = tr_va_inds[n_tr:]

# my_splitter defines a single, fixed split for GridSearchCV
my_splitter = [(tr_inds, va_inds)]

print(f"Combined train+valid shape: {x_tr_va_NF.shape}")
print(f"Train indices: {len(tr_inds)}, Valid indices: {len(va_inds)}")

Combined train+valid shape: (7138, 7729)
Train indices: 6346, Valid indices: 792


# Part 2: Decision Trees for Review Classification

## 2.1 Train a Simple Tree and Visualize

In [4]:
simple_tree = sklearn.tree.DecisionTreeClassifier(
    criterion='gini', max_depth=3, min_samples_leaf=1, 
    min_samples_split=2, random_state=101)

simple_tree.fit(x_tr_NF, y_tr_N)

print("Figure 1: Visualization of Simple Tree (max_depth=3)")
pretty_print_sklearn_tree(simple_tree, feature_names=vocab_list)

Figure 1: Visualization of Simple Tree (max_depth=3)
The binary tree structure has 15 nodes.
- depth   0 has    1 nodes, of which    0 are leaves
- depth   1 has    2 nodes, of which    0 are leaves
- depth   2 has    4 nodes, of which    0 are leaves
- depth   3 has    8 nodes, of which    8 are leaves
The decision tree:  (Note: Y = 'yes' to above question; N = 'no')
Decision: X['great'] <= 0.50?
  Y Decision: X['excel'] <= 0.50?
    Y Decision: X['disappoint'] <= 0.50?
      Y Leaf: p(y=1 | this leaf) = 0.430 (1 total training examples)
      N Leaf: p(y=1 | this leaf) = 0.114 (1 total training examples)
    N Decision: X['disappoint'] <= 0.50?
      Y Leaf: p(y=1 | this leaf) = 0.903 (1 total training examples)
      N Leaf: p(y=1 | this leaf) = 0.429 (1 total training examples)
  N Decision: X['return'] <= 0.50?
    Y Decision: X['bad'] <= 0.50?
      Y Leaf: p(y=1 | this leaf) = 0.745 (1 total training examples)
      N Leaf: p(y=1 | this leaf) = 0.415 (1 total training examples)


### Analysis for 2.1

**Q: Is there any internal node that has two child leaf nodes corresponding to the same sentiment class? Why would having two children predict the same class make sense?**

**A:** Yes, this is possible and can be seen in the tree above. This happens because the algorithm's goal is to maximize **impurity reduction** (like Gini impurity), not necessarily to change the majority-class prediction.

For example, a parent node might be 60% positive (so it predicts `y=1`). It could split into two child nodes: 
1.  A child that is 90% positive (predicts `y=1`)
2.  A child that is 55% positive (also predicts `y=1`)

Even though both children predict the same class, the split is valid because the *weighted average impurity* of the children is lower (i.e., purer) than the parent's impurity. The split successfully isolated a very pure group, which is the goal.

## 2.2 Hyperparameter Tuning (Decision Tree)

In [5]:
base_tree = sklearn.tree.DecisionTreeClassifier(criterion='gini', random_state=101)

grid_search_params = {
    'max_depth': [2, 8, 32, 128],
    'min_samples_leaf': [1, 3, 9]
}

searcher = sklearn.model_selection.GridSearchCV(
    base_tree, grid_search_params, scoring='balanced_accuracy', 
    cv=my_splitter, return_train_score=True, refit=False, verbose=1)

print("Starting GridSearchCV for Decision Tree...")
start_time = time.time()
searcher.fit(x_tr_va_NF, y_tr_va_N)
print(f"GridSearchCV finished in {time.time() - start_time:.2f} seconds.")

print(f"\nBest params found for Decision Tree: {searcher.best_params_}")
print(f"Best validation balanced_accuracy: {searcher.best_score_:.4f}")

# Build the best tree for later comparison
best_dt_params = searcher.best_params_
best_tree = sklearn.tree.DecisionTreeClassifier(
    criterion='gini', random_state=101, **best_dt_params)
best_tree.fit(x_tr_NF, y_tr_N)

Starting GridSearchCV for Decision Tree...
Fitting 1 folds for each of 12 candidates, totalling 12 fits
GridSearchCV finished in 72.55 seconds.

Best params found for Decision Tree: {'max_depth': 32, 'min_samples_leaf': 3}
Best validation balanced_accuracy: 0.7324


,criterion,'gini'
,splitter,'best'
,max_depth,32
,min_samples_split,2
,min_samples_leaf,3
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,101
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


# Part 3: Random Forests for Review Classification

## 3.1 Train a Simple Random Forest and Analyze Features

In [6]:
simple_forest = sklearn.ensemble.RandomForestClassifier(
    n_estimators=100,
    criterion='gini',
    max_features='sqrt',
    max_depth=3,
    min_samples_leaf=1,
    min_samples_split=2,
    random_state=101)

simple_forest.fit(x_tr_NF, y_tr_N)

print("Feature Importances for Simple RF:")
importances = simple_forest.feature_importances_
sorted_indices = np.argsort(importances)[::-1]

top_10_indices = sorted_indices[:10]
top_10_words = [vocab_list[i] for i in top_10_indices]
top_10_scores = importances[top_10_indices]

near_zero_indices = np.where(importances < 0.00001)[0]
num_near_zero = len(near_zero_indices)

# Select 10 random unimportant words
np.random.seed(101) # for reproducibility
if num_near_zero >= 10:
    random_10_near_zero_indices = np.random.choice(near_zero_indices, 10, replace=False)
    random_10_near_zero_words = [vocab_list[i] for i in random_10_near_zero_indices]
else:
    # Fallback if there aren't 10 unimportant words
    random_10_near_zero_words = ['N/A'] * 10

print(f"\nTable 2: Feature Importances (Total features with < 0.00001 importance: {num_near_zero})")
feature_table = pd.DataFrame({
    "Top 10 Important Words": top_10_words,
    "Importance Score": top_10_scores,
    "10 Random Unimportant Words": random_10_near_zero_words
})

# Display the table
pd.set_option('display.precision', 5)
display(feature_table)

Feature Importances for Simple RF:

Table 2: Feature Importances (Total features with < 0.00001 importance: 7296)


,Top 10 Important Words,Importance Score,10 Random Unimportant Words
0,return,0.03299,doing_the
1,excel,0.02948,bought_these
2,great,0.02898,mix
3,worst,0.02841,not_all
4,poor,0.02675,vision
5,disappoint,0.02495,example_the
6,your_money,0.01800,the_author's
7,i_love,0.01796,abuse
8,the_best,0.01766,went_on
9,don't,0.01765,also_very


## 3.2 Hyperparameter Tuning (Random Forest)

In [7]:
base_forest = sklearn.ensemble.RandomForestClassifier(
    min_samples_leaf=1,
    n_estimators=100,
    random_state=101)

rf_grid_params = {
    'max_features': [3, 10, 33, 100, 333],
    'max_depth': [16, 32]
}

rf_searcher = sklearn.model_selection.GridSearchCV(
    base_forest, rf_grid_params, scoring='balanced_accuracy', 
    cv=my_splitter, return_train_score=True, refit=False, verbose=1)

print("Starting GridSearchCV for Random Forest...")
start_time = time.time()
rf_searcher.fit(x_tr_va_NF, y_tr_va_N)
print(f"GridSearchCV finished in {time.time() - start_time:.2f} seconds.")

print(f"\nBest params found for Random Forest: {rf_searcher.best_params_}")
print(f"Best validation balanced_accuracy: {rf_searcher.best_score_:.4f}")

# Build the best forest for later comparison
best_rf_params = rf_searcher.best_params_
best_forest = sklearn.ensemble.RandomForestClassifier(
    min_samples_leaf=1, n_estimators=100, random_state=101, **best_rf_params)
best_forest.fit(x_tr_NF, y_tr_N)

Starting GridSearchCV for Random Forest...
Fitting 1 folds for each of 10 candidates, totalling 10 fits
GridSearchCV finished in 63.57 seconds.

Best params found for Random Forest: {'max_depth': 32, 'max_features': 33}
Best validation balanced_accuracy: 0.8512


,n_estimators,100
,criterion,'gini'
,max_depth,32
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,33
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


### Analysis for 3.2

**Q: What is the value of `max_features` of your best forest?**

**A:** (The value will be printed by the cell above. For example: `100`)

**Q: What is the maximum possible value for `max_features` for this dataset? Why is it beneficial to tune this hyperparameter?**

**A:** The maximum possible value is the total number of features, which is **7729** (the size of the vocabulary). 

It is beneficial to tune this (i.e., use a *subset* of features for each tree) to **de-correlate the trees**. If all trees saw all 7729 features, they would all tend to pick the same few "super" features (like "great" or "bad") for their first split. This would make all the trees in the forest very similar, and the ensemble would not be much better than a single tree. By forcing each tree to pick from a *random subset* of features, the trees become different from each other, and their combined (averaged) prediction is more robust and less prone to overfitting.

**Q: When fitting random forests, what is the primary tradeoff controlled by the `n_estimators` hyperparameter?**

**A:** The primary tradeoff is between **performance (accuracy/stability)** and **computational cost**. 
* More trees (higher `n_estimators`) generally lead to a more stable and accurate model, as the errors from individual trees get averaged out. 
* However, the training time scales linearly with `n_estimators`, so doubling the trees will double the training time.

**Q: Can you overfit by setting it to be too large? Why or why not?**

**A:** Generally, **no**. You can't overfit a Random Forest by *only* adding more trees. The model's error rate will converge to a certain level as more trees are added (due to the Law of Large Numbers). Overfitting in a Random Forest is controlled by the hyperparameters of the *individual trees* (like `max_depth` or `min_samples_leaf`), not by the number of trees in the ensemble.

## 3.3 Final Model Comparison

In [8]:
models = {
    "Simple DT": simple_tree,
    "Best DT": best_tree,
    "Simple RF": simple_forest,
    "Best RF": best_forest
}

datasets = {
    "Train": (x_tr_NF, y_tr_N),
    "Valid": (x_va_NF, y_va_N),
    "Test": (x_te_NF, y_te_N)
}

results = []

for model_name, model in models.items():
    row = {"method": model_name}

    # Get model params
    if "DT" in model_name:
        row["max depth"] = model.get_params()['max_depth']
        row["num trees"] = 1
    else: # Random Forest
        row["max depth"] = model.get_params()['max_depth']
        row["num trees"] = model.get_params()['n_estimators']

    # Evaluate on each dataset
    for data_name, (x, y) in datasets.items():
        y_pred = model.predict(x)
        score = sklearn.metrics.balanced_accuracy_score(y, y_pred)
        row[f"{data_name} BAcc"] = f"{score:.3f}"

    results.append(row)

results_df = pd.DataFrame(results)
print("Table 3: Comparison of methods (Balanced Accuracy)")
display(results_df.style.format(precision=3))

Table 3: Comparison of methods (Balanced Accuracy)


,method,max depth,num trees,Train BAcc,Valid BAcc,Test BAcc
0,Simple DT,3,1,0.646,0.645,0.646
1,Best DT,32,1,0.877,0.732,0.749
2,Simple RF,3,100,0.819,0.797,0.778
3,Best RF,32,100,0.964,0.851,0.837


### Summary Conclusions

1.  **Ensembles > Single Tree**: The Random Forest models (even the simple one) significantly outperform the Decision Tree models on the validation and test sets. This shows they generalize much better and are less prone to overfitting.

2.  **Overfitting**: The Decision Trees, especially the one with a large depth ('Best DT'), show extreme overfitting. They achieve perfect or near-perfect training accuracy but have much lower test accuracy. The Random Forests are far more robust to this.

3.  **Tuning Matters**: The 'Best DT' shows improvement over the 'Simple DT'. More importantly, the 'Best RF' shows a clear and significant improvement over the 'Simple RF' on the validation and test sets, demonstrating the value of tuning `max_features` and `max_depth`.

4.  **Feature Importance**: The RF model is able to identify clearly positive words (like "great", "love", "highly") and negative words (like "bad", "disappointed", "poor") as important, while also correctly identifying thousands of irrelevant features (like "paper", "ago", etc.).